In [3]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits, ghz_circuits
from mitiq import MeasurementResult, Observable, PauliString, raw
from functools import partial

n_qubits = 2

circuit = generate_rb_circuits(2, 10)[0]

def execute(circuit: cirq.Circuit, noise_level: float = 0.002, p0: float = 0.05) -> MeasurementResult:
    """Execute a circuit with depolarizing noise of strength ``noise_level`` and readout errors ...
    """
    measurements = circuit[-1]
    circuit =  circuit[:-1]
    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=10000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

obs = Observable(PauliString("ZI"), PauliString("IZ"))


# no QEM
noisy = raw.execute(circuit, execute, obs)
ideal = raw.execute(circuit, partial(execute, noise_level=0, p0=0), obs)
print("Ideal value:", "{:.5f}".format(ideal.real))
print("Unmitigated value:", "{:.5f}".format(noisy.real))



# REM
from mitiq import rem

p0 = p1 = 0.05
icm = rem.generate_inverse_confusion_matrix(2, p0, p1)
rem_executor = rem.mitigate_executor(execute, inverse_confusion_matrix=icm)

rem_result = obs.expectation(circuit, rem_executor)
print("Mitigated value obtained with REM:", "{:.5f}".format(rem_result.real))



# ZNE
from mitiq import zne

zne_executor = zne.mitigate_executor(execute, observable=obs, scale_noise=zne.scaling.folding.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))



# Stack ZNE on top of REM
rem_zne_executor = zne.mitigate_executor(rem_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

rem_zne_result = rem_zne_executor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(rem_zne_result.real))



# NEW STUFF: stack REM on ZNE
zne_rem_executor = rem.mitigate_executor(zne_executor, inverse_confusion_matrix=icm)

zne_rem_result = zne_rem_executor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(zne_rem_result.real))


Ideal value: 2.00000
Unmitigated value: 1.40380
Mitigated value obtained with REM: 1.61440
Mitigated value obtained with ZNE: 1.84360
Mitigated value obtained with REM + ZNE: 2.01380


TypeError: Result is not of type MeasurementResult.